In [134]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path 
import socket
import pickle
import scipy.io as sio
#from rich import inspect

Set data path

In [135]:
this_computer_name = socket.gethostname()
print(this_computer_name)

DESKTOP-F5LCT4Q


In [136]:

if this_computer_name == 'DESKTOP-V3D343U':
    freemocap_validation_data_path = Path(r"I:\My Drive\HuMoN_Research_Lab\FreeMoCap_Stuff\FreeMoCap_Balance_Validation\data")
elif this_computer_name == 'DESKTOP-F5LCT4Q':
    freemocap_validation_data_path = Path(r"C:\Users\aaron\Documents\HumonLab\Spring2022\ValidationStudy\FreeMocap_Data")
    #freemocap_validation_data_path = Path(r'D:\freemocap2022\FreeMocap_Data')
else:
    freemocap_validation_data_path = Path(r"C:\Users\kiley\Documents\HumonLab\SampleFMC_Data\FreeMocap_Data-20220216T173514Z-001\FreeMocap_Data")

In [137]:
## Build'n'Load Stuff (paths and arrays)
#freemocap_validation_data_path = Path(r"C:\Users\aaron\Documents\HumonLab\Spring2022\ValidationStudy\FreeMocap_Data") #path to your data folder





sessionID = 'session_SER_1_20_22' #name of the sessionID folder
this_freemocap_session_path = freemocap_validation_data_path / sessionID
this_freemocap_data_path = this_freemocap_session_path/'DataArrays'
mediapipe_data_path = this_freemocap_data_path / 'mediaPipeSkel_3d_smoothed.npy'
qualisys_data_path = this_freemocap_data_path/'qualisysData_3d.mat'


In [138]:
qualysis_mat_file = sio.loadmat(qualisys_data_path)
qualisys_pose_data = qualysis_mat_file['skeleton_fr_mar_dim_reorg']

qual_num_frames = qualisys_pose_data.shape[0]

#qual_num_frames = 8000

qual_num_frame_range = range(qual_num_frames)



f=2

In [95]:

mediapipeSkel_fr_mar_dim = np.load(mediapipe_data_path) #loads in the data as a numpy array

In [96]:
num_pose_joints = 33 #number of pose joints tracked by mediapipe 




pose_joint_range = range(num_pose_joints)
#frame_range = range(first_frame,last_frame)

mediapipe_pose_data = mediapipeSkel_fr_mar_dim[:,0:num_pose_joints,:] #load just the pose joints into a data array, removing hands and face data 

num_frames = len(mediapipe_pose_data)

num_frame_range = range(num_frames)

skel_x = mediapipe_pose_data[:,:,0]
skel_y = mediapipe_pose_data[:,:,1]
skel_z = mediapipe_pose_data[:,:,2]

![](WinterAnthropometryTable.jpeg?raw=true)

In [139]:

#segments is a list of the body segments that we want the center of mass for. joint_connections is the corresponding joints that make up that segment in order of [proximal joint,distal joint]
# segment_COM_lengths is a corresponding list of center of mass lengths for each segment, and segment_COM_percentages is a corresponding list of the percentage of total body COM each segment takes up. 
segments = [
'head',
'trunk',
'right_upper_arm',
'left_upper_arm',
'right_forearm',
'left_forearm',
'right_hand',
'left_hand',
'right_thigh',
'left_thigh',
'right_shin',
'left_shin',
'right_foot',
'left_foot'
]

joint_connections = [
[],
['mid_chest_marker', 'mid_hip_marker'], #do these joint_connections have to correspond to mediapipe? yes
['right_shoulder','right_elbow'],
['left_shoulder','left_elbow'],
['right_elbow', 'right_wrist'],
['left_elbow', 'left_wrist'],
['right_wrist', 'right_hand_marker'], #need to spend some time on the hands,can there be more than two elements in these lists?
['left_wrist', 'left_hand_marker'],
['right_hip', 'right_knee'],
['left_hip', 'left_knee'],
['right_knee', 'right_ankle'],
['left_knee', 'left_ankle'],
['right_back_of_foot_marker', 'right_foot_index'], #will need to figure out these naming conventions later
['left_back_of_foot_marker', 'left_foot_index']
]

segment_COM_lengths = [
.5,
.4310,
.5772,
.5772,
.4574,
.4574,
.7900, #check on the hand, did you actually use this in the com approx? Yes I did
.7900, #check on the hand, did you actually use this in the com approx? Yes I did
.4095,
.4095,
.4459,
.4459,
.4415, #check on the foot, did you actually use this in the com approx? No, I didn't
.4415  #check on the foot, did you actually use this in the com approx? No, I didn't
]

segment_COM_percentages = [
.0694,
.4346,
.0271,
.0271,
.0162,
.0162,
.0061,
.0061,
.1416,
.1416,
.0433,
.0433,
.0137,
.0137
]

num_segments = len(segments)

In [140]:

#create a table with the joint connections, COM length, and percentage for each segment into a dataframe
df = pd.DataFrame(list(zip(segments,joint_connections,segment_COM_lengths,segment_COM_percentages)),columns = ['Segment Name','Joint Connection','Segment COM Length','Segment COM Percentage'])

#create a new copy of this dataframe where the index is set to be the body segment (as opposed to 0,1,2 etc.)
segment_conn_len_perc_dataframe = df.set_index('Segment Name')

segment_conn_len_perc_dataframe


,Joint Connection,Segment COM Length,Segment COM Percentage
Segment Name,,,
head,[],0.5000,0.0694
trunk,"[mid_chest_marker, mid_hip_marker]",0.4310,0.4346
right_upper_arm,"[right_shoulder, right_elbow]",0.5772,0.0271
left_upper_arm,"[left_shoulder, left_elbow]",0.5772,0.0271
right_forearm,"[right_elbow, right_wrist]",0.4574,0.0162
left_forearm,"[left_elbow, left_wrist]",0.4574,0.0162
right_hand,"[right_wrist, right_hand_marker]",0.7900,0.0061
left_hand,"[left_wrist, left_hand_marker]",0.7900,0.0061
right_thigh,"[right_hip, right_knee]",0.4095,0.1416


In [43]:
def build_mediapipe_skeleton(mediapipe_pose_data,segment_dataframe):
    """ This function takes in the mediapipe pose data array and the segment_conn_len_perc_dataframe. 
        For each frame of data, it loops through each segment we want to find and identifies the names
        of the proximal and distal joints of that segment. Then it searches the mediapipe_indices list
        to find the index that corresponds to the name of that segment. We plug the index into the 
        mediapipe_pose_data array to find the proximal/distal joints' XYZ coordinates at that frame. 
        The segment, its proximal joint and its distal joint gets thrown into a dictionary. 
        And then that dictionary gets saved to a list for each frame. By the end of the function, you 
        have a list that contains the skeleton segment XYZ coordinates for each frame."""

    def build_mediapipe_virtual_markers(mediapipe_indices,frame):
        def mediapipe_index_finder(list_of_joint_names):

            indices = []
            for name in list_of_joint_names:
                this_name_index = mediapipe_indices.index(name)
                indices.append(this_name_index)
            
            return indices
        
        def mediapipe_XYZ_finder(indices_list):

                XYZ_coordinates = []
                for index in indices_list:
                    this_joint_coordinate = mediapipe_pose_data[frame,index,:]
                    XYZ_coordinates.append(this_joint_coordinate)

                return XYZ_coordinates

        # def build_virtual_hand_marker(hand_joint_connection, mediapipe_indices):

        #         hand_indices = mediapipe_index_finder(hand_joint_connection)

        #         hand_XYZ_coordinates = mediapipe_XYZ_finder(hand_indices)

        #         wrist = hand_XYZ_coordinates[0]
        #         index = hand_XYZ_coordinates[1]
        #         pinky = hand_XYZ_coordinates[2]
                
        #         # hand_virtual_marker = index/2

        #         knuckle_width = pinky - index
        #         third_metacarple = index + knuckle_width/3
                
        #         hand_virtual_marker = third_metacarple - wrist

                # return hand_virtual_marker
            
        def build_virtual_trunk_marker(trunk_joint_connection,mediapipe_indices):
                trunk_indices = mediapipe_index_finder(trunk_joint_connection)

                trunk_XYZ_coordinates = mediapipe_XYZ_finder(trunk_indices)

                trunk_proximal = (trunk_XYZ_coordinates[0] + trunk_XYZ_coordinates[1])/2
                trunk_distal = (trunk_XYZ_coordinates[2] + trunk_XYZ_coordinates[3])/2

                return trunk_proximal, trunk_distal
        
        def build_virtual_foot_marker(foot_joint_connection, mediapipe_indices):
                foot_indices = mediapipe_index_finder(foot_joint_connection)

                foot_XYZ_coordinates = mediapipe_XYZ_finder(foot_indices)

                foot_virtual_marker = (foot_XYZ_coordinates[0] + foot_XYZ_coordinates[1])/2

                return foot_virtual_marker

        #left_hand_joint_connection = ['left_wrist', 'left_index', 'left_pinky']   
        #right_hand_joint_connection = ['right_wrist', 'right_index', 'right_pinky']
        trunk_joint_connection = ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip']

        left_foot_joint_connection = ['left_heel','left_ankle']
        right_foot_joint_connection = ['right_heel','right_ankle']

        #left_hand_virtual_marker = build_virtual_hand_marker(left_hand_joint_connection,mediapipe_indices)
        #right_hand_virtual_marker = build_virtual_hand_marker(right_hand_joint_connection,mediapipe_indices)
        trunk_virtual_markers = build_virtual_trunk_marker(trunk_joint_connection,mediapipe_indices)
        left_foot_virtual_marker = build_virtual_foot_marker(left_foot_joint_connection,mediapipe_indices)
        right_foot_virtual_marker = build_virtual_foot_marker(right_foot_joint_connection,mediapipe_indices)
    
        return trunk_virtual_markers, left_foot_virtual_marker, right_foot_virtual_marker

    #this is a list of the 33 pose joints that mediapipe tracks
    mediapipe_indices = [
    'nose',
    'right_eye_inner',
    'right_eye',
    'right_eye_outer',
    'left_eye_inner',
    'left_eye',
    'left_eye_outer',
    'right_ear',
    'left_ear',
    'mouth_right',
    'mouth_left',
    'right_shoulder',
    'left_shoulder',
    'right_elbow',
    'left_elbow',
    'right_wrist',
    'left_wrist',
    'right_pinky',
    'left_pinky',
    'right_index',
    'left_index',
    'right_thumb',
    'left_thumb',
    'right_hip',
    'left_hip',
    'right_knee',
    'left_knee',
    'right_ankle',
    'left_ankle',
    'right_heel',
    'left_heel',
    'right_foot_index',
    'left_foot_index'
    ]
    

    mediapipe_frame_segment_joint_XYZ = [] #empty list to hold all the skeleton XYZ coordinates/frame
    for frame in num_frame_range: #NOTE: need to change frame_range to numFrames 2/09/2022 - AC

        trunk_virtual_markers, left_foot_virtual_marker, right_foot_virtual_marker = build_mediapipe_virtual_markers(mediapipe_indices,frame)

        mediapipe_pose_skeleton_coordinates = {}
        for segment,segment_info in segment_conn_len_perc_dataframe.iterrows(): #iterate through the data frame by the segment name and all the info for that segment
            if segment == 'trunk':

                #based on index, excract coordinate data from fmc mediapipe data
                mediapipe_pose_skeleton_coordinates[segment] = [trunk_virtual_markers[0], 
                                                                trunk_virtual_markers[1]
                                                                ]
            elif segment == 'left_hand' or segment == 'right_hand':
                proximal_joint_hand = segment_info['Joint Connection'][0]
                if segment == 'left_hand':
                    distal_joint_hand = 'left_index'
                else:
                    distal_joint_hand = 'right_index'

                proximal_joint_hand_index = mediapipe_indices.index(proximal_joint_hand)
                distal_joint_hand_index = mediapipe_indices.index(distal_joint_hand)

                mediapipe_pose_skeleton_coordinates[segment] = [mediapipe_pose_data[frame,proximal_joint_hand_index, :],
                                                                mediapipe_pose_data[frame,distal_joint_hand_index,:]]
            elif segment == 'left_foot' or segment == 'right_foot':
                if segment == 'left_foot':
                    proximal_joint_foot_XYZ = left_foot_virtual_marker
                else:
                    proximal_joint_foot_XYZ = right_foot_virtual_marker
                distal_joint_foot = segment_info['Joint Connection'][1]
                distal_joint_foot_index = mediapipe_indices.index(distal_joint_foot)
                mediapipe_pose_skeleton_coordinates[segment] = [proximal_joint_foot_XYZ,
                                                                mediapipe_pose_data[frame, distal_joint_foot_index,:]]
            # elif segment == 'right_hand':
            #     proximal_joint_hand = segment_info['Joint Connection'][0]
            #     distal_joint_hand = right_hand_virtual_marker

            #     proximal_joint_hand_index = mediapipe_indices.index(proximal_joint_hand)

            #     mediapipe_pose_skeleton_coordinates[segment] = [mediapipe_pose_data[frame,proximal_joint_hand_index, :],
            #                                                     right_hand_virtual_marker]
            
            
            # elif segment in ('left_hand', 'right_hand'): #segment == 'right_foot' or 'left_foot' or 'right_hand' or 'left_hand':
            #     #assign joint connectino to segment
            #     proximal_joint_hand = segment_info['Joint Connection'][0]
            #     distal_joint_hand = 

                
            #     #assign index
            #     proximal_joint_hand_index = mediapipe_indices.index(proximal_joint_hand)
            #     distal_joint1_hand_index = mediapipe_indices.index(distal_joint1_hand)
            #     distal_joint2_hand_index = mediapipe_indices.index(distal_joint2_hand)
                
            #     #based on index, extract coordinate data from fmc mediapipe data
            #     mediapipe_pose_skeleton_coordinates[segment] = [mediapipe_pose_data[frame,proximal_joint_hand_index, :],
            #                                                     mediapipe_pose_data[frame,distal_joint1_hand_index, :],
            #                                                     mediapipe_pose_data[frame,distal_joint2_hand_index, :]
            #                                                     ]
                
            else:
                proximal_joint_name = segment_info['Joint Connection'][0] 
                distal_joint_name = segment_info['Joint Connection'][1]

            #get the mediapipe index for the proximal and distal joint for this segment
                proximal_joint_index = mediapipe_indices.index(proximal_joint_name)
                distal_joint_index = mediapipe_indices.index(distal_joint_name)

            #use the mediapipe indices to get the XYZ coordinates for the prox/distal joints and throw it in a dictionary
            #mediapipe_pose_skeleton_coordinates[segment] = {'proximal':mediapipe_pose_data[frame,proximal_joint_index,:],'distal':mediapipe_pose_data[frame,distal_joint_index,:]}
                mediapipe_pose_skeleton_coordinates[segment] = [mediapipe_pose_data[frame,proximal_joint_index,:],mediapipe_pose_data[frame,distal_joint_index,:]]
                
        mediapipe_frame_segment_joint_XYZ.append(mediapipe_pose_skeleton_coordinates)
        f = 2 
    
    return mediapipe_frame_segment_joint_XYZ

    f= 2
skelcoordinates_frame_segment_joint_XYZ = build_mediapipe_skeleton(mediapipe_pose_data,segment_conn_len_perc_dataframe)
 
#print(skelcoordinates_frame_segment_joint_XYZ[0])
skeleton_file_name = this_freemocap_data_path/'mediapipeSkelcoordinates_frame_segment_joint_XYZ.pkl'

open_file = open(skeleton_file_name, "wb")
pickle.dump(skelcoordinates_frame_segment_joint_XYZ, open_file)
open_file.close()
f=2 



IndexError: list index out of range

In [141]:
def build_qualisys_skeleton(qualisys_pose_data,segment_conn_len_perc_dataframe):
    def qualisys_index_finder(list_of_joint_names, qualisys_indices):
        indices = []
        for name in list_of_joint_names:
            this_name_index = qualisys_indices.index(name)
            indices.append(this_name_index)
        return indices

    def qualisys_XYZ_finder(indices_list,qualisys_pose_data):

            XYZ_coordinates = []
            for index in indices_list:
                this_joint_coordinate = qualisys_pose_data[frame,index,:]
                XYZ_coordinates.append(this_joint_coordinate)

            return XYZ_coordinates

    def build_mid_chest_marker(mid_chest_joints,qualisys_indices,segment_dataframe):
        f=2
        chest_indices = qualisys_index_finder(mid_chest_joints, qualisys_indices)

        chest_coordinates = qualisys_XYZ_finder(chest_indices,qualisys_pose_data)

        mid_chest_marker = (chest_coordinates[0] + chest_coordinates[1])/2

        return mid_chest_marker


    qualisys_indices = [ #these indices have been renamed to match the joint connections list above
    'mid_hip_marker',
    'Spine',
    'Spine1',
    'Spine2',
    'Neck',
    'head',
    'LeftShoulder_false',
    'left_shoulder',
    'left_elbow',
    'left_wrist',
    'left_hand_marker',
    'RightShoulder_false',
    'right_shoulder',
    'right_elbow',
    'right_wrist',
    'right_hand_marker',
    'left_hip',
    'left_knee',
    'left_back_of_foot_marker',
    'left_foot_index',
    'right_hip',
    'right_knee',
    'right_back_of_foot_marker',
    'right_foot_index'
    ]

    mid_chest_joints = ['left_shoulder','right_shoulder']

    qualisys_frame_segment_joint_XYZ = [] #empty list to hold all the skeleton XYZ coordinates/frame
    for frame in qual_num_frame_range: 
        qualisys_pose_skeleton_coordinates = {}
        for segment,segment_info in segment_conn_len_perc_dataframe.iterrows():
            if segment == 'head':
                head_index = qualisys_indices.index(segment)
                qualisys_pose_skeleton_coordinates[segment] = [qualisys_pose_data[frame,head_index,:]]
            
            elif segment == 'trunk':

                mid_chest_marker_XYZ = build_mid_chest_marker(mid_chest_joints,qualisys_indices,segment_conn_len_perc_dataframe)
                
                mid_hip_marker_name = segment_info['Joint Connection'][1]
                mid_hip_marker_index = qualisys_indices.index(mid_hip_marker_name)
                mid_hip_marker_XYZ = qualisys_pose_data[frame,mid_hip_marker_index,:]

                qualisys_pose_skeleton_coordinates[segment] = [mid_chest_marker_XYZ,mid_hip_marker_XYZ]


            else:
                proximal_joint_name = segment_info['Joint Connection'][0] 
                distal_joint_name = segment_info['Joint Connection'][1]

                if 'ankle' in distal_joint_name: #qualysis doesn't have an ankle joint like mediapipe does, so need to replace the ankle joint with the foot markers they do track 
                    if segment == 'right_shin':
                        distal_joint_name = 'right_back_of_foot_marker'
                    else:
                        distal_joint_name = 'left_back_of_foot_marker'

            #get the mediapipe index for the proximal and distal joint for this segment
                proximal_joint_index = qualisys_indices.index(proximal_joint_name)
                distal_joint_index = qualisys_indices.index(distal_joint_name)
                qualisys_pose_skeleton_coordinates[segment] = [qualisys_pose_data[frame,proximal_joint_index,:],qualisys_pose_data[frame,distal_joint_index,:]]
        qualisys_frame_segment_joint_XYZ.append(qualisys_pose_skeleton_coordinates)

    return qualisys_frame_segment_joint_XYZ

qual_skelcoordinates_frame_segment_joint_XYZ = build_qualisys_skeleton(qualisys_pose_data,segment_conn_len_perc_dataframe)
skeleton_file_name = this_freemocap_data_path/'qualisysSkelcoordinates_frame_segment_joint_XYZ.pkl'

open_file = open(skeleton_file_name, "wb")
pickle.dump(qual_skelcoordinates_frame_segment_joint_XYZ, open_file)
open_file.close()

f=2

In [3]:
    # qualisys__og_indices = [
    # 'Hips',
    # 'Spine',
    # 'Spine1',
    # 'Spine2',
    # 'Neck',
    # 'Head',
    # 'LeftShoulder',
    # 'LeftArm',
    # 'LeftForeArm',
    # 'LeftForeArmRoll',
    # 'LeftHand',
    # 'RightShoulder',
    # 'RightArm',
    # 'RightForeArm',
    # 'RightForeArmRoll',
    # 'RightHand',
    # 'LeftUpLeg',
    # 'LeftLeg',
    # 'LeftFoot',
    # 'LeftToeBase',
    # 'RightUpLeg',
    # 'RightLeg',
    # 'RightFoot',
    # 'RightToeBase'
    # ]

    # name = qualisys__og_indices.index('LeftShoulder')
    # print(name)

6


In [10]:
# def build_mediapipe_virtual_markers(mediapipe_indices,frame):

#     def mediapipe_index_finder(list_of_joint_names):

#         indices = []
#         for name in list_of_joint_names:
#             this_name_index = mediapipe_indices.index(name)
#             indices.append(this_name_index)
#         return indices
    
#     def mediapipe_XYZ_finder(indices_list):

#             XYZ_coordinates = []

#             for index in indices_list:
#                 this_joint_coordinate = mediapipe_pose_data[frame,index,:]
            
#             XYZ_coordinates.append(this_joint_coordinate)
#             return XYZ_coordinates

#     def build_virtual_hand_marker(hand_joint_connection, mediapipe_indices):

#             hand_indices = mediapipe_index_finder(hand_joint_connection)

#             hand_XYZ_coordinates = mediapipe_XYZ_finder(hand_indices)

#             wrist = hand_XYZ_coordinates[0]
#             index = hand_XYZ_coordinates[1]
#             pinky = hand_XYZ_coordinates[2]

#             knuckle_width = pinky - index
#             third_metacarple = index + knuckle_width/3
            
#             hand_virtual_marker = third_metacarple - wrist
#             return hand_virtual_marker
        
#     def build_virtual_trunk_marker(trunk_joint_connection,mediapipe_indices):
#             trunk_indices = mediapipe_index_finder(trunk_joint_connection)

#             trunk_XYZ_coordinates = mediapipe_XYZ_finder(trunk_indices)

#             trunk_proximal = (trunk_XYZ_coordinates[0] + trunk_XYZ_coordinates[1])/2
#             trunk_distal = (trunk_XYZ_coordinates[2] + trunk_XYZ_coordinates[3])/2
#             return trunk_proximal, trunk_distal

#     left_hand_joint_connection = ['left_wrist', 'left_index', 'left_pinky']   
#     right_hand_joint_connection = ['right_wrist', 'right_index', 'right_pinky']
    
#     trunk_joint_connection = ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip']

#     left_hand_virtual_marker = build_virtual_hand_marker(left_hand_joint_connection,mediapipe_indices)
#     right_hand_virtual_marker = build_virtual_hand_marker(right_hand_joint_connection,mediapipe_indices)


In [142]:
def calculate_segment_COM(segment_conn_len_perc_dataframe,skelcoordinates_frame_segment_joint_XYZ, num_frame_range):
    segment_COM_frame_dict = []
    for frame in num_frame_range:
        segment_COM_dict = {}
        for segment,segment_info in segment_conn_len_perc_dataframe.iterrows():
            this_segment_XYZ = skelcoordinates_frame_segment_joint_XYZ[frame][segment]

            if segment == 'head': #since qualisys doesn't have a firm head calculating thing, just using the 
                segment_COM_dict[segment] = this_segment_XYZ[0]
            else:
                this_segment_proximal = this_segment_XYZ[0]
                this_segment_distal = this_segment_XYZ[1]
                this_segment_COM_length = segment_info['Segment COM Length']

                this_segment_COM = this_segment_proximal + this_segment_COM_length*(this_segment_distal-this_segment_proximal)
                segment_COM_dict[segment] = this_segment_COM
        segment_COM_frame_dict.append(segment_COM_dict)
    return segment_COM_frame_dict
    f = 2


def reformat_segment_COM(segment_COM_frame_dict, num_frame_range):
    
    segment_COM_frame_imgPoint_XYZ = np.empty([int(len(num_frame_range)),int(num_segments),3])
    for frame in num_frame_range:
        this_frame_skeleton = segment_COM_frame_dict[frame]
        for joint_count,segment in enumerate(this_frame_skeleton.keys()):
            segment_COM_frame_imgPoint_XYZ[frame,joint_count,:] = this_frame_skeleton[segment]
    return segment_COM_frame_imgPoint_XYZ


#segment_COM_frame_dict = calculate_segment_COM(segment_conn_len_perc_dataframe, skelcoordinates_frame_segment_joint_XYZ)
#segment_COM_frame_imgPoint_XYZ = reformat_segment_COM(segment_COM_frame_dict)
#np.save(this_freemocap_data_path/'segmentedCOM_frame_joint_XYZ.npy',segment_COM_frame_imgPoint_XYZ)

qualisys_segment_COM_frame_dict = calculate_segment_COM(segment_conn_len_perc_dataframe, qual_skelcoordinates_frame_segment_joint_XYZ, qual_num_frame_range)
qualisys_segment_COM_frame_imgPoint_XYZ = reformat_segment_COM(qualisys_segment_COM_frame_dict, qual_num_frame_range)
np.save(this_freemocap_data_path/'qualisys_segmentedCOM_frame_joint_XYZ.npy',qualisys_segment_COM_frame_imgPoint_XYZ)

f=2

            



In [143]:
def calculate_total_body_COM(segment_conn_len_perc_dataframe,segment_COM_frame_dict,num_frame_range):
    totalBodyCOM_frame_XYZ = np.empty([int(len(num_frame_range)),3])

    for frame in num_frame_range:

        this_frame_total_body_percentages = []
        this_frame_skeleton = segment_COM_frame_dict[frame]

        for segment, segment_info in segment_conn_len_perc_dataframe.iterrows():

            this_segment_COM = this_frame_skeleton[segment]
            this_segment_COM_percentage = segment_info['Segment COM Percentage']

            this_segment_total_body_percentage = this_segment_COM * this_segment_COM_percentage
            this_frame_total_body_percentages.append(this_segment_total_body_percentage)

        this_frame_total_body_COM = np.nansum(this_frame_total_body_percentages,axis = 0)
       
        totalBodyCOM_frame_XYZ[frame,:] = this_frame_total_body_COM

    f=2
    return totalBodyCOM_frame_XYZ        
    
# totalBodyCOM_frame_XYZ = calculate_total_body_COM(segment_conn_len_perc_dataframe,segment_COM_frame_dict)
# np.save(this_freemocap_data_path/'totalBodyCOM_frame_XYZ.npy',totalBodyCOM_frame_XYZ)

qualisys_totalBodyCOM_frame_XYZ = calculate_total_body_COM(segment_conn_len_perc_dataframe,qualisys_segment_COM_frame_dict,qual_num_frame_range)
np.save(this_freemocap_data_path/'qualisys_totalBodyCOM_frame_XYZ.npy',qualisys_totalBodyCOM_frame_XYZ)
f = 2

In [139]:
good_frame = 3000
goodframe_x = skel_x[good_frame,:]
goodframe_y = skel_y[good_frame,:]
goodframe_z = skel_z[good_frame,:]

segment_COM_x = segment_COM_frame_imgPoint_XYZ[good_frame,:,0]
segment_COM_y = segment_COM_frame_imgPoint_XYZ[good_frame,:,1]
segment_COM_z = segment_COM_frame_imgPoint_XYZ[good_frame,:,2]

total_COM_x = totalBodyCOM_frame_XYZ[good_frame,0]
total_COM_y = totalBodyCOM_frame_XYZ[good_frame,1]
total_COM_z = totalBodyCOM_frame_XYZ[good_frame,2]

f=2



def calculate_total_COM()

In [140]:
plot_this_frame = good_frame


plot_frame_bones_XYZ = skelcoordinates_frame_segment_joint_XYZ[plot_this_frame]

%matplotlib
#plt.ion
figure = plt.figure()
ax = figure.add_subplot(111, projection = '3d')

ax.scatter(goodframe_x, goodframe_y,goodframe_z)

#bonez
for segment in plot_frame_bones_XYZ.keys():
    prox_joint = plot_frame_bones_XYZ[segment][0] 
    dist_joint = plot_frame_bones_XYZ[segment][1]
    
    bone_x,bone_y,bone_z = [prox_joint[0],dist_joint[0]],[prox_joint[1],dist_joint[1]],[prox_joint[2],dist_joint[2]] 

    ax.plot(bone_x,bone_y,bone_z,color = 'black')
    
ax.scatter(segment_COM_x,segment_COM_y,segment_COM_z, color = 'orange')    

ax.scatter(total_COM_x,total_COM_y,total_COM_z, color = 'purple')
plt.show()
f = 2







Using matplotlib backend: <object object at 0x00000181671F4F70>


In [ ]:
# %matplotlib
# #plt.ion()
# figure = plt.figure()
# ax = figure.add_subplot(111, projection = '3d')
# ax.scatter(goodframe_x, goodframe_y,goodframe_z)
# ax.scatter(segment_COM_x,segment_COM_y,segment_COM_z)
# plt.show()
# # inspect(ax,methods= True)
# f=2

Using matplotlib backend: <object object at 0x00000256B7FC4C90>


In [ ]:
# #http://www.kdm.p.lodz.pl/articles/2017/3/21_3_4.pdf
# r_upperarm_COM_x = goodframe_x[r_upperarm[0]] + (.5772)*(goodframe_x[r_upperarm[1]]-goodframe_x[r_upperarm[0]])
# l_upperarm_COM_x = goodframe_x[l_upperarm[0]] + (.5772)*(goodframe_x[l_upperarm[1]]-goodframe_x[l_upperarm[0]])

# r_forearm_COM_x = goodframe_x[r_forearm[0]] + (.4574)*(goodframe_x[r_forearm[1]]-goodframe_x[r_forearm[0]])
# l_forearm_COM_x = goodframe_x[l_forearm[0]] + (.4574)*(goodframe_x[l_forearm[1]]-goodframe_x[l_forearm[0]])

# r_upperarm_COM_y = goodframe_y[r_upperarm[0]] + (.5772)*(goodframe_y[r_upperarm[1]]-goodframe_y[r_upperarm[0]])
# l_upperarm_COM_y = goodframe_y[l_upperarm[0]] + (.5772)*(goodframe_y[l_upperarm[1]]-goodframe_y[l_upperarm[0]])

# r_forearm_COM_y = goodframe_y[r_forearm[0]] + (.4574)*(goodframe_y[r_forearm[1]]-goodframe_y[r_forearm[0]])
# l_forearm_COM_y = goodframe_y[l_forearm[0]] + (.4574)*(goodframe_y[l_forearm[1]]-goodframe_y[l_forearm[0]])

# segment_com_x = [r_upperarm_COM_x,l_upperarm_COM_x,r_forearm_COM_x,l_forearm_COM_x]
# segment_com_y = [r_upperarm_COM_y,l_upperarm_COM_y,r_forearm_COM_y,l_forearm_COM_y]

